#MVP Engenharia de Dados - 2025
Thomas A. S. Abrantes


In [0]:
###-------------------------------------------------------------
### REMOÇÃO DE ARQUIVO NO DBFS
###-------------------------------------------------------------

#display(dbutils.fs.ls('dbfs:/FileStore/imdb/'))
#dbutils.fs.rm("dbfs:/FileStore/imdb/title_akas_4-1.tsv")

###Introdução

O que desejo consumir na base de dados IMDB
- Todos os gêneros existentes na base
- Titulos com um único gênero
- Lista de Gêneros que complementam títulos Adultos
- Títulos com nova geração de atores com menos de 30 anos
- Ator que mais participou de títulos que estão cadastrados no catalogo
- Ator com menos participações nos títulos cadastrados no catalogo
- Atores que nasceram antes de 1970
- Quantidade de títulos lançados por Década (70, 80, 90, etc.)


#Dicionário de dados

Fonte: A base de dados utilizada foi extraída do IMDB (https://datasets.imdbws.com/)

Esta base consiste nos arquivos abaixo no formato tsv e de acordo com a documentação disponivilizada pelo IMDBWS, A primeira linha em cada arquivo contém cabeçalhos que descrevem o que há em cada coluna. Um '\N' indica que um campo específico está faltando ou é nulo para aquele um título/nome.

###title.akas.tsv
Arquivo contendo os títulos AKAs, que são Títulos Alternativos. 
São os nomes diferentes que um filme pode ter, em diferentes idiomas e países.

O arquivo consiste nas colunas:

- titleId (string): Identificador no formato alfanumérico exclusivo do título.
- ordering (inteiro) – É um número para identificar exclusivamente as linhas para um determinado titleId. Neste arquivo há mais de uma linha com mesmo titleId.
- title (string) – Nome do título.
- region (string) - Descreve a região, com dois caracteres. 
> Exemplo: US para United States, JP para Japão na versão do título.
- language (string) - o idioma do título.
- types (string) -Conjunto enumerado em forma de string de atributos para este título alternativo. 
> - Na documentação é indicado como matriz mas na importação esses valores vão como uma string contendo todos os valores da coluna e caso seja usado será tratado como uma lista. De acordo com a documentação pode conter os valores abaixo mas destaca que pode haver novos valores podem ser adicionados no futuro:
> - alternative
> - dvd
> - festival
> - tv
> - video
> - working
> - original
> - imdbDisplay
> - attributes (string) - Termos adicionais para descrever este título alternativo.
- isOriginalTitle (inteiro) – O Valor 0 (false) indica que o título não original e 1 (true) que o título é original.

>  Na documentação é indicado como matriz mas na importação esses valores vão como uma string contendo todos os valores da coluna e caso seja usado será tratado como uma lista.

###title.basics.tsv
Arquivo contendo as informações básicas dos títulos.
O arquivo consiste nas colunas:

- tconst (string) - Identificador no formato alfanumérico exclusivo do título.
- titleType (string) – Informa o tipo/formato do título.
> Exemplo: filme, curta, série de TV, episódio de TV, vídeo, etc. 
> 
No arquivo esses valores estão registrados em inglês como movie, short, tvseries. tvepisode, video.

- primaryTitle (string) – Nome do título popularmente conhecido ou usado pelos cineastas em materiais promocionais no seu  lançamento.
- originalTitle (string) - Titulo original, no idioma original.
- isAdult (boolean) - O valor 0(false) informa que o título não é somente para o público adulto e 1(true) informa que título é destinado ao público adulto.
- startYear (inteiro com 4 caracteres) - Ano de lançamento do título. Caso seja uma série de TV o valor é o ano da primeira temporada da ´serie.
- endYear (inteiro com 4 caracteres) - Ano de término para série de TV. Para outros títulos esse valor vem preenchido com “\N”.
- runtimeMinutes - Tempo, em minutos, de execução do título principal.
- genres (matriz de strings) – Pode conter até três gêneros associados ao título.
> Esse campos quando houver mais de uma gênero, sera exibido separado por uma vírgula.
> Exemplo: "Documentary,Short".

###title.crew.tsv.
Arquivo que contém informações da equipe de filmagem de um filme ou programa de TV.
O arquivo consiste nas colunas:

- tconst  (string) - Identificador no formato alfanumérico exclusivo do título.
- directors  (matriz de nconsts no formato string) - Identificador no formato alfanumérico identificando o(s) diretore(s) do título.
- writers  (matriz de nconsts no formato string) - Identificador no formato alfanumérico identificando o(s) escritore(s) do título.

###tile.episode.tsv
Arquivo que contém informações de um episódio de uma série de um programa de TV.
O arquivo consiste nas colunas:

- tconst (string) - identificador alfanumérico do episódio.
- parentTconst (string) - identificador alfanumérico da série de TV de Origem (pai).
- seasonNumber (inteiro) – Número da temporada do episódio.
- episodeNumber (inteiro) – Número do episódio com valor tconst em uma série de TV.

###title.principals.tsv

> Observação: Devido ao seu tamanho maior que 1GB foi particionado em 6 partes, e unificado apos a carga dos dados no Notebook no Databricks.
> 
Arquivo que contém informações do título original de uma obra, na sua língua original.
O arquivo consiste nas colunas:

- tconst (string) - Identificador alfanumérico exclusivo do título.
- ordering (inteiro) – É um número para identificar exclusivamente as linhas para um determinado titleId.
- nconst (string) - Identificador alfanumérico exclusivo do nome/pessoa.
- category (string) - Categoria do trabalho em que a pessoa executou na produção.
- job (string) - Cargo específico, se aplicável, caso contrário '\N'.
- characters (string) - Nome do personagem interpretado, se aplicável, caso contrário '\N'.

###title.ratings.tsv
Arquivo que contém informações sobre as classificações e o número de votos para cada título no site.
O arquivo consiste nas colunas:

- tconst (string) - Identificador alfanumérico exclusivo do título.
- averageRating – Valor da média ponderada de todas as avaliações individuais dos usuários.
- numVotes (inteiro) - Número de votos que o título recebeu.

###name.basics.tsv
Arquivo que contém informações báscias de uma pessoa.
O arquivo consiste nas colunas:

- nconst (string) - identificador alfanumérico exclusivo do nome/pessoa.
- primaryName (string) – nome pelo qual a pessoa é mais frequentemente creditada.
- birthYear (inteiro) – Ano de nascimento no formato AAAA (4 dígitos).
- deathYear (inteiro) – Ano de falecimento no formato AAAA (4 dígitos). se aplicável, caso contrário '\N'.
- primaryProfession (matriz de strings) – Valor das três principais profissões da pessoa.
- knownForTitles (matriz de tconsts) – Valores dos títulos pelos quais a pessoa é conhecida.


### Setup para rodar o notebook com os dados
- Imports das bibliotecas para consumo dos dados.
- Link para carga de dados do IMDB.

In [0]:
#bibioteca para uso do comando com Pandas
import pandas as pd

#biblioteca para carga de dados e request dos dados
import io
import requests

#link para consulta direto no site do IMDB
imdb_url = "https://datasets.imdbws.com/"

##Importação dos dados do IMDB
Utilizando a base de acesso publico no endereço https://datasets.imdbws.com/ que disponibiliza os arquivos no formato TSV compactado no padrão GZIP:

- name.basics.tsv.gz
- title.akas.tsv.gz
- title.basics.tsv.gz
- title.crew.tsv.gz
- title.episode.tsv.gz
- title.principals.tsv.gz
- title.ratings.tsv.gz


###Consultar os arquivos carregados no DBFS

In [0]:
display(dbutils.fs.ls('dbfs:/FileStore/imdb/'))

path,name,size,modificationTime
dbfs:/FileStore/imdb/name_basics.tsv,name_basics.tsv,878052986,1742904843000
dbfs:/FileStore/imdb/title_akas_1.tsv,title_akas_1.tsv,616099140,1742910239000
dbfs:/FileStore/imdb/title_akas_2.tsv,title_akas_2.tsv,618917721,1742910236000
dbfs:/FileStore/imdb/title_akas_3.tsv,title_akas_3.tsv,632519935,1742910526000
dbfs:/FileStore/imdb/title_akas_4.tsv,title_akas_4.tsv,775056138,1742910610000
dbfs:/FileStore/imdb/title_basics.tsv,title_basics.tsv,996687208,1742905106000
dbfs:/FileStore/imdb/title_crew.tsv,title_crew.tsv,380029823,1742904985000
dbfs:/FileStore/imdb/title_episode.tsv,title_episode.tsv,232652143,1742905039000
dbfs:/FileStore/imdb/title_principals_1.tsv,title_principals_1.tsv,914305463,1742908769000
dbfs:/FileStore/imdb/title_principals_2.tsv,title_principals_2.tsv,658047612,1742908703000


#Carga dos Dados

###Arquivo "name.basics.tsv.gz"

In [0]:
###-------------------------------------------------------------
### -- carga de dados a partir do site do IMDB
###-------------------------------------------------------------
##imdb_gz = imdb_url + "name.basics.tsv.gz"
#df_imdb = pd.read_csv(#imdb_gz, compression='gzip', sep='\t')
#df_imdb.head()
#-------------------------------------------------------------

###-------------------------------------------------------------
### -- carga a partir do DBFS
###-------------------------------------------------------------
df_name_basics = spark.read.csv('dbfs:/FileStore/imdb/name_basics.tsv', sep='\t', header=True)


###Arquivo "title.basics.tsv.gz".

In [0]:
###-------------------------------------------------------------
### -- carga de dados a partir do site do IMDB
###-------------------------------------------------------------
##imdb_gz = imdb_url + "title.basics.tsv.gz"
#df_imdb = pd.read_csv(#imdb_gz, compression='gzip', sep='\t')
#df_imdb.head()

###-------------------------------------------------------------
### -- carga a partir do DBFS
###-------------------------------------------------------------
df_title_basics = spark.read.csv('dbfs:/FileStore/imdb/title_basics.tsv', sep='\t', header=True)


###Arquivo "title.crew.tsv.gz".

In [0]:
###-------------------------------------------------------------
### -- carga de dados a partir do site do IMDB
###-------------------------------------------------------------
##imdb_gz = imdb_url + "title.crew.tsv.gz"
#df_imdb = pd.read_csv(#imdb_gz, compression='gzip', sep='\t')
#df_imdb.head()

###-------------------------------------------------------------
### -- carga a partir do DBFS
###-------------------------------------------------------------
df_title_crew = spark.read.csv('dbfs:/FileStore/imdb/title_crew.tsv', sep='\t', header=True)


###Arquivo "title.episode.tsv.gz".

In [0]:
###-------------------------------------------------------------
### -- carga de dados a partir do site do IMDB
###-------------------------------------------------------------
##imdb_gz = imdb_url + "title.episode.tsv.gz"
#df_imdb = pd.read_csv(#imdb_gz, compression='gzip', sep='\t')
#df_imdb.head()

###-------------------------------------------------------------
### -- carga a partir do DBFS
###-------------------------------------------------------------
df_title_episode = spark.read.csv('dbfs:/FileStore/imdb/title_episode.tsv', sep='\t', header=True)
df_title_episode.display()


tconst,parentTconst,seasonNumber,episodeNumber
tt0031458,tt32857063,\N,\N
tt0041951,tt0041038,1,9
tt0042816,tt0989125,1,17
tt0042889,tt0989125,\N,\N
tt0043426,tt0040051,3,42
tt0043631,tt0989125,2,16
tt0043693,tt0989125,2,8
tt0043710,tt0989125,3,3
tt0044093,tt0959862,1,6
tt0044668,tt0044243,2,16


###Arquivo "title.ratings.tsv.gz".

In [0]:
###-------------------------------------------------------------
### -- carga de dados a partir do site do IMDB
###-------------------------------------------------------------
##imdb_gz = imdb_url + "title.ratings.tsv.gz"
#df_imdb = pd.read_csv(#imdb_gz, compression='gzip', sep='\t')
#df_imdb.head()

###-------------------------------------------------------------
### -- carga a partir do DBFS
###-------------------------------------------------------------
df_title_rattings = spark.read.csv('dbfs:/FileStore/imdb/title_ratings.tsv', sep='\t', header=True)


###Arquivo "title.akas.tsv.gz".

In [0]:
###-------------------------------------------------------------
### -- carga de dados a partir do site do IMDB
###-------------------------------------------------------------
#imdb_gz = imdb_url + "title.akas.tsv.gz"
#df_imdb = pd.read_csv(imdb_gz, compression='gzip', sep='\t')
#df_imdb.head()

###-------------------------------------------------------------
### -- carga a partir do DBFS
### OBSERVAÇÃO:  O DATABRICKS nao PERMITE carregar mais de 1GB 
### particionei o arquivo então diminui a massa de dados do arquivo para realizar a carga
###-------------------------------------------------------------
df_akas1 = spark.read.csv('dbfs:/FileStore/imdb/title_akas_1.tsv', sep='\t', header=True)
df_akas2 = spark.read.csv('dbfs:/FileStore/imdb/title_akas_2.tsv', sep='\t', header=True)
df_akas3 = spark.read.csv('dbfs:/FileStore/imdb/title_akas_3.tsv', sep='\t', header=True)
df_akas4 = spark.read.csv('dbfs:/FileStore/imdb/title_akas_4.tsv', sep='\t', header=True)

###Arquivos com origem do "title_principals.tsv.gz"

- title_principals_1.tsv
- title_principals_2.tsv
- title_principals_3.tsv
- title_principals_4.tsv

In [0]:
###-------------------------------------------------------------
### -- carga de dados a partir do site do IMDB
###-------------------------------------------------------------
##imdb_gz = imdb_url + "title.principals.tsv.gz"
#df_imdb = pd.read_csv(#imdb_gz, compression='gzip', sep='\t')
#df_imdb.head()

###-------------------------------------------------------------
### -- carga a partir do DBFS
###-------------------------------------------------------------
### -- carga a partir do DBFS
### OBSERVAÇÃO:  O DATABRICKS nao PERMITE carregar mais de 1GB 
### dividi o arquivo "title_principals.tsv" em partes de ate 1gb para carregar e posteriormente fazer um UNION
###-------------------------------------------------------------
df_title_principals1 = spark.read.csv('dbfs:/FileStore/imdb/title_principals_1.tsv', sep='\t', header=True)
df_title_principals2 = spark.read.csv('dbfs:/FileStore/imdb/title_principals_2.tsv', sep='\t', header=True)
df_title_principals3 = spark.read.csv('dbfs:/FileStore/imdb/title_principals_3.tsv', sep='\t', header=True)
df_title_principals4 = spark.read.csv('dbfs:/FileStore/imdb/title_principals_4.tsv', sep='\t', header=True)
df_title_principals5 = spark.read.csv('dbfs:/FileStore/imdb/title_principals_5.tsv', sep='\t', header=True)
df_title_principals6 = spark.read.csv('dbfs:/FileStore/imdb/title_principals_6.tsv', sep='\t', header=True)

#Camada BRONZE

Criando as tabelas com arquivos importados.
Criação do banco de dados (Schema) IMDB_BRONZE importando os dados dos arquivos para tabelas com nomes contendo prefixo "tb" e nome do arquivo de origem nos seus respectivos Data Frames.

In [0]:
%sql
-- Removendo Schema para recriar
DROP SCHEMA IF EXISTS IMDB_DB_BRONZE CASCADE;

-- CRIANDO O BANCO DE DADOS (schema)
CREATE SCHEMA IMDB_DB_BRONZE;

###Criando e preenchendo a "**tb_name_basics**".

In [0]:
#Removendo vínculo do arquivo da tabela
dbutils.fs.rm("dbfs:/user/hive/warehouse/imdb_db_bronze.db/tb_name_basics/", True)

Out[12]: True

In [0]:
%sql
-- Criando a tabela "tb_name_basics"
CREATE OR REPLACE TABLE IMDB_DB_BRONZE.tb_name_basics (
  nconst STRING,
  primaryName STRING,
  birthYear STRING,
  deathYear STRING,
  primaryProfession STRING,
  knownForTitles STRING
)

In [0]:
df_name_basics.write.format("delta").mode("overwrite").saveAsTable("IMDB_DB_BRONZE.tb_name_basics")

###Criando e preenchendo a "**tb_title_basics**".

In [0]:
#Removendo vínculo do arquivo da tabela
dbutils.fs.rm("dbfs:/user/hive/warehouse/imdb_db_bronze.db/tb_title_basics/", True)

Out[15]: True

In [0]:
%sql
-- Criando  a tabela "tb_title_basics"
CREATE OR REPLACE TABLE IMDB_DB_BRONZE.tb_title_basics (
  tconst STRING,
  titleType STRING,
  primaryTitle STRING,
  originalTitle STRING,
  isAdult STRING,
  startYear STRING,
  endYear STRING,
  runtimeMinutes STRING,
  genres STRING
)

In [0]:
df_title_basics.write.format("delta").mode("overwrite").saveAsTable("IMDB_DB_BRONZE.tb_title_basics")

###Criando e preenchendo a "**tb_title_crew**".

In [0]:
#Removendo vínculo do arquivo da tabela
dbutils.fs.rm("dbfs:/user/hive/warehouse/imdb_db_bronze.db/tb_title_crew/", True)

Out[18]: True

In [0]:
%sql
-- Criando a tabela "tb_title_crew"
CREATE OR REPLACE TABLE IMDB_DB_BRONZE.tb_title_crew (
  tconst STRING,
  directors STRING,
  writers STRING
)

In [0]:
df_title_crew.write.format("delta").mode("overwrite").saveAsTable("IMDB_DB_BRONZE.tb_title_crew")

###Criando e preenchendo a "**tb_title_episode**".

In [0]:
#Removendo vínculo do arquivo da tabela
dbutils.fs.rm("dbfs:/user/hive/warehouse/imdb_db_bronze.db/tb_title_episode/", True)

Out[22]: True

In [0]:
%sql
-- Criando a tabela "tb_title_episode"
CREATE OR REPLACE TABLE IMDB_DB_BRONZE.tb_title_episode (
  tconst STRING,
  parentTconst STRING,
  seasonNumber STRING,
  episodeNumber STRING
)

In [0]:
df_title_episode.write.format("delta").mode("overwrite").saveAsTable("IMDB_DB_BRONZE.tb_title_episode")

###Criando e preenchendo a "**tb_title_akas**"

In [0]:
#Removendo vínculo do arquivo da tabela
dbutils.fs.rm("dbfs:/user/hive/warehouse/imdb_db_bronze.db/tb_title_akas/", True)

Out[25]: True

In [0]:
%sql
-- Criando a tabela "tb_title_akas"
CREATE OR REPLACE TABLE IMDB_DB_BRONZE.tb_title_akas (
    titleId STRING,
    ordering STRING,
    title STRING,
    region STRING,
    language STRING,
    types STRING,
    attributes STRING,
    isOriginalTitle STRING
)

Criando um dataframe com a união dos DataFrames dos arquivos **"title_akas_[numero].tsv"**.

In [0]:
df_akas = df_akas1.union(df_akas2) \
    .union(df_akas3) \
        .union(df_akas4)

Preenchendo a "tb_title_akas" com o DataFrame unificado.

In [0]:
df_akas.write.format("delta").mode("overwrite").saveAsTable("IMDB_DB_BRONZE.tb_title_akas")

###Criando e preenchendo a "**tb_title_principals**".

In [0]:
#Removendo vínculo do arquivo da tabela
dbutils.fs.rm("dbfs:/user/hive/warehouse/imdb_db_bronze.db/tb_title_principals/", True)

Out[29]: False

In [0]:
%sql
-- Criando a tabela "tb_title_principals"
CREATE OR REPLACE TABLE IMDB_DB_BRONZE.tb_title_principals (
  tconst STRING,
  ordering STRING,
  nconst STRING,
  category STRING,
  job STRING,
  characters STRING
)

Criando um dataframe com a união dos DataFrames dos arquivos **"title_principals_[numero].tsv"**.

In [0]:
df_title_principals = df_title_principals1.union(df_title_principals2) \
    .union(df_title_principals3) \
        .union(df_title_principals4) \
            .union(df_title_principals5) \
                .union(df_title_principals6)
                

Preenchendo a "tb_title_pricipals" com o DataFrame unificado.

In [0]:
df_title_principals.write.format("delta").mode("overwrite").saveAsTable("IMDB_DB_BRONZE.tb_title_principals")

Criando e preenchendo a "tb_title_ratings".

In [0]:
#Removendo vínculo do arquivo da tabela
dbutils.fs.rm("dbfs:/user/hive/warehouse/imdb_db_bronze.db/tb_title_ratings/", True)

In [0]:
%sql
CREATE OR REPLACE TABLE IMDB_DB_BRONZE.tb_title_ratings (
  tconst STRING,
  averageRating STRING,
  numVotes STRING
)

In [0]:
df_title_rattings.write.format("delta").mode("overwrite").saveAsTable("IMDB_DB_BRONZE.tb_title_ratings")